In [1]:
from binance.client import Client
import talib
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import time

In [2]:
# Replace with your API key and secret
api_key = "MYgUFxFMjeVglbzUsips2x38QR11XNJQ5NuYVbov3QBITOcyYmnBGA3MtuHTegWY"
api_secret = "tOfsgUFcmjb1odifv2F1z4gYjl8Fvbzr5xjbQmiMbNSZnADjRcpCuToHp1H0R2x7"
client = Client(api_key=api_key, api_secret=api_secret)
symbol = "BTCUSDT"  # Replace with the trading pair of your choice

# Initialize start and end times
start_time = None
end_time = None

# Fetch data in batches
all_klines = []
while len(all_klines) < 2880:
    # Set start and end times for the current batch
    if start_time is None:
        start_time = int(time.time() * 1000) - 2880 * 5 * 60 * 1000  # 2880 * 5 minutes
    else:
        start_time = end_time + 1
    end_time = start_time + 1000 * 5 * 60 * 1000 - 1  # 1000 * 5 minutes

    # Fetch the current batch of klines
    klines = client.futures_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_5MINUTE, limit=1000, startTime=start_time, endTime=end_time)
    all_klines.extend(klines)

# Extract close prices from the fetched klines
close_prices = np.array([float(entry[4]) for entry in all_klines])
low_prices = np.array([float(entry[3]) for entry in all_klines])
high_prices = np.array([float(entry[2]) for entry in all_klines])

In [20]:
# Replace with your API key and secret
api_key = "MYgUFxFMjeVglbzUsips2x38QR11XNJQ5NuYVbov3QBITOcyYmnBGA3MtuHTegWY"
api_secret = "tOfsgUFcmjb1odifv2F1z4gYjl8Fvbzr5xjbQmiMbNSZnADjRcpCuToHp1H0R2x7"
client = Client(api_key=api_key, api_secret=api_secret)
symbol = "BTCUSDT"  # Replace with the trading pair of your choice

# 獲取當前時間
current_time = int(time.time() * 1000)

# 計算所需的時間範圍
start_time = current_time - 2880 * 5 * 60 * 1000  # 2880 * 5 分鐘

# 準備資料列表
close_prices = []
low_prices = []
high_prices = []
timestamps = []

# 循環獲取數據
while len(close_prices) < 2880:
    # 設置當前批次的開始和結束時間
    if len(close_prices) == 0:
        batch_start_time = start_time
    else:
        batch_start_time = batch_end_time + 1
    batch_end_time = batch_start_time + 1000 * 5 * 60 * 1000 - 1  # 1000 * 5 分鐘

    # 獲取當前批次的 kline 數據
    klines = client.futures_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_5MINUTE, limit=1000, startTime=batch_start_time, endTime=batch_end_time)

    # 提取需要的數據
    for entry in klines:
        timestamps.append(int(entry[0]))
        close_prices.append(float(entry[4]))
        low_prices.append(float(entry[3]))
        high_prices.append(float(entry[2]))

# initialize data of lists.
data = {'timestamps': timestamps,
        'Close': close_prices,
        'High': high_prices,
        'Low': low_prices}

df = pd.DataFrame(data)  
df['timestamps'] = pd.to_datetime(df['timestamps'], unit='ms') + pd.Timedelta(hours=8)
df

,timestamps,Close,High,Low
0,2024-05-19 22:20:00,67090.0,67117.5,67083.7
1,2024-05-19 22:25:00,66934.2,67092.8,66911.1
2,2024-05-19 22:30:00,66993.7,67011.6,66934.2
3,2024-05-19 22:35:00,66952.5,66993.7,66915.9
4,2024-05-19 22:40:00,66884.9,66952.6,66852.6
...,...,...,...,...
2875,2024-05-29 21:55:00,67921.6,67970.0,67839.7
2876,2024-05-29 22:00:00,67916.3,67947.0,67845.9
2877,2024-05-29 22:05:00,67933.1,68007.6,67910.1
2878,2024-05-29 22:10:00,67935.1,67988.0,67876.4


In [21]:
# 指標設定
macd_fast = 12
macd_slow = 26 
macd_signal = 9

ema_fast = 3
ema_slow = 21
ema_signal = 8
ema_short = 7

# 計算指標
df['MACD'], df['MACD_Signal'], df['MACD_Hist'] = talib.MACD(df['Close'], fastperiod=macd_fast, slowperiod=macd_slow, signalperiod=macd_signal)
df['EMA_Fast'] = talib.EMA(df['Close'], timeperiod=ema_fast)
df['EMA_Slow'] = talib.EMA(df['Close'], timeperiod=ema_slow)
df['EMA_Signal'] = talib.EMA(df['Close'], timeperiod=ema_signal)
df['EMA_Short'] = talib.EMA(df['Close'], timeperiod=ema_short)
df['ParaSAR'] = talib.SAR(df['High'], df['Low'], acceleration=0.02, maximum=0.2)

# 進場條件
df['Signal'] = 0
df.loc[(df['MACD'] > df['MACD_Signal']) & (df['ParaSAR'] < df['Close']) & (df['Close'] > df['EMA_Short']) & (df['Close'].shift(1) < df['EMA_Short'].shift(1)), 'Signal'] = 1
df.loc[(df['MACD'] < df['MACD_Signal']) & (df['ParaSAR'] > df['Close']) & (df['Close'] < df['EMA_Short']) & (df['Close'].shift(1) > df['EMA_Short'].shift(1)), 'Signal'] = -1

# 交易執行
df['Entry_Price'] = np.where(df['Signal'] == 1, df['Close'] + 10, np.where(df['Signal'] == -1, df['Close'] - 10, 0))
df['Stop_Loss'] = np.where(df['Signal'] == 1, df['Close'] - 20, np.where(df['Signal'] == -1, df['Close'] + 20, 0))
df['Take_Profit'] = np.where(df['Signal'] == 1, df['Entry_Price'] + (df['Entry_Price'] - df['Stop_Loss']) * 2, np.where(df['Signal'] == -1, df['Entry_Price'] - (df['Entry_Price'] - df['Stop_Loss']) * 2, 0))

# 退出策略
df.loc[(df['MACD'] < df['MACD_Signal']) | (df['Close'] < df['EMA_Short']), 'Signal'] = 0

# 收益計算
df['Returns'] = np.log(df['Close'] / df['Close'].shift(1))
df['Strategy_Returns'] = df['Signal'].shift(1) * df['Returns']

In [22]:
df

,timestamps,Close,High,Low,MACD,MACD_Signal,MACD_Hist,EMA_Fast,EMA_Slow,EMA_Signal,EMA_Short,ParaSAR,Signal,Entry_Price,Stop_Loss,Take_Profit,Returns,Strategy_Returns
0,2024-05-19 22:20:00,67090.0,67117.5,67083.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,0.0,NaN,NaN
1,2024-05-19 22:25:00,66934.2,67092.8,66911.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67117.500000,0,0.0,0.0,0.0,-0.002325,-0.0
2,2024-05-19 22:30:00,66993.7,67011.6,66934.2,NaN,NaN,NaN,67005.966667,NaN,NaN,NaN,67113.372000,0,0.0,0.0,0.0,0.000889,0.0
3,2024-05-19 22:35:00,66952.5,66993.7,66915.9,NaN,NaN,NaN,66979.233333,NaN,NaN,NaN,67109.326560,0,0.0,0.0,0.0,-0.000615,-0.0
4,2024-05-19 22:40:00,66884.9,66952.6,66852.6,NaN,NaN,NaN,66932.066667,NaN,NaN,NaN,67105.362029,0,0.0,0.0,0.0,-0.001010,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,2024-05-29 21:55:00,67921.6,67970.0,67839.7,-14.904329,-11.792467,-3.111862,67900.536339,67907.502569,67891.296323,67890.065674,68180.892463,0,0.0,0.0,0.0,-0.000472,-0.0
2876,2024-05-29 22:00:00,67916.3,67947.0,67845.9,-12.293473,-11.892669,-0.400804,67908.418169,67908.302335,67896.852695,67896.624255,68147.256915,0,0.0,0.0,0.0,-0.000078,-0.0
2877,2024-05-29 22:05:00,67933.1,68007.6,67910.1,-8.767663,-11.267667,2.500005,67920.759085,67910.556669,67904.907652,67905.743192,68115.639500,0,0.0,0.0,0.0,0.000247,0.0
2878,2024-05-29 22:10:00,67935.1,67988.0,67876.4,-5.745815,-10.163297,4.417482,67927.929542,67912.787881,67911.617063,67913.082394,68085.919130,0,0.0,0.0,0.0,0.000029,0.0


In [23]:
df.to_excel("df.xlsx")

In [175]:
ema_32 = talib.EMA(close_prices, timeperiod=32)
ema_128 = talib.EMA(close_prices, timeperiod=256)
rsi = talib.RSI(close_prices, timeperiod=14)
upper_band, middle_band, lower_band = talib.BBANDS(close_prices, timeperiod=15, nbdevup=2, nbdevdn=2)
macd, signal, _ = talib.MACD(close_prices, fastperiod=12, slowperiod=26, signalperiod=9)
slowk, slowd = talib.STOCH(high_prices, low_prices, close_prices, 
                            fastk_period=14, 
                            slowk_period=9, 
                            slowk_matype=6,
                            slowd_period=9,
                            slowd_matype=6)
fastk, fastd = talib.STOCHRSI(close_prices, 
                            timeperiod=14,
                            fastk_period=5, 
                            fastd_period=3,
                            fastd_matype=0)
adx = talib.ADX(high_prices, low_prices, close_prices, timeperiod=14)
atr = talib.ATR(high_prices, low_prices, close_prices, timeperiod=14)
cci = talib.CCI(high_prices, low_prices, close_prices, timeperiod=14)
willr = talib.WILLR(high_prices, low_prices, close_prices, timeperiod=14)
roc = talib.ROC(close_prices, timeperiod=12)

In [176]:
def crossover(ema_short,ema_long):

    # Golden Crossover
    golden_crossover = np.where((ema_short[:-1] < ema_long[:-1]) & (ema_short[1:] >= ema_long[1:]))[0]

    # Death Crossover 
    death_crossover = np.where((ema_short[:-1] > ema_long[:-1]) & (ema_short[1:] <= ema_long[1:]))[0]
    
    return golden_crossover, death_crossover

In [178]:
golden_crossover, death_crossover = crossover(ema_32, ema_128)

In [185]:
# 創建 Plotly 圖表
fig = go.Figure()

# 設置圖表大小
fig.update_layout(
    width=1400,
    height=700,
    xaxis_title="Time",
    yaxis_title="Price",
    legend_title_text="Legend",
    # 增加內邊距
    margin=dict(l=50, r=50, t=50, b=50),
    # 調整背景色彩
    plot_bgcolor='#f8f8f8',
    # 優化網格線
    xaxis_gridcolor='#e1e1e1',
    yaxis_gridcolor='#e1e1e1',
    xaxis_gridwidth=1,
    yaxis_gridwidth=1,
    # 調整圖例位置和樣式
    legend=dict(
        x=0.85,
        y=1,
        font_size=12,
        bgcolor='#f8f8f8',
        bordercolor='#e1e1e1',
        borderwidth=1
    ),
    # 調整標題和軸標籤
    title_font_size=16,
    xaxis_tickfont_size=12,
    yaxis_tickfont_size=12
)

# 添加 close prices 曲線
fig.add_trace(go.Scatter(
    x=list(range(len(close_prices))),
    y=close_prices,
    mode='lines',
    name='Closing Prices',
    line=dict(color='#FFA500'),
    hovertemplate='Close: %{y:.2f}<extra></extra>'
))

# 添加上 upper 布林線
fig.add_trace(go.Scatter(
    x=list(range(2880)),
    y=upper_band,
    mode='lines',
    name='Upper Bollinger Band',
    line=dict(color='#808080'),
    hovertemplate='Upper Band: %{y:.2f}<extra></extra>'
))

# 添加上 upper 布林線
fig.add_trace(go.Scatter(
    x=list(range(2880)),
    y=middle_band,
    mode='lines',
    name='Middle Bollinger Band',
    line=dict(color='#808080'),
    hovertemplate='Middle Band: %{y:.2f}<extra></extra>'
))

# 添加下 lower 布林線
fig.add_trace(go.Scatter(
    x=list(range(2880)),
    y=lower_band,
    mode='lines',
    name='Lower Bollinger Band',
    line=dict(color='#808080'),
    hovertemplate='Lower Band: %{y:.2f}<extra></extra>'
))

# 將所有指標放在同一個 trace 中
fig.add_trace(go.Scatter(
    x=list(range(len(close_prices))),
    y=close_prices,
    mode='markers',
    marker=dict(
        color='#FFA500',
        size=0.5
    ),
    name='Close',
    text=[f"Close: {c:.2f}<br>EMA (32): {e32:.2f}<br>EMA (128): {e128:.2f}<br>RSI: {r:.2f}<br>MACD: {m:.2f}<br>Stoch. Slow K: {sk:.2f}<br>Stoch. Slow D: {sd:.2f}<br>Stoch. Fast K: {fk:.2f}<br>Stoch. Fast D: {fd:.2f}<br>ADX: {adx:.2f}<br>ATR: {atr:.2f}<br>CCI: {cci:.2f}<br>Williams %R: {willr:.2f}<br>ROC: {roc:.2f}" for c, e32, e128, r, m, sk, sd, fk, fd, adx, atr, cci, willr, roc in zip(close_prices, ema_32, ema_128, rsi, macd, slowk, slowd, fastk, fastd, adx, atr, cci, willr, roc)],
    hovertemplate='%{text}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=golden_crossover,
    y=[close_prices[i] for i in golden_crossover],
    mode='markers',
    marker=dict(
        color='red',
        size=8
    ),
    name='Golden Crossover'
))

fig.add_trace(go.Scatter(
    x=death_crossover,
    y=[close_prices[i] for i in death_crossover],
    mode='markers',
    marker=dict(
        color='green',
        size=8
    ),
    name='Death Crossover'
))

fig.show()
fig.write_html("plotly_chart.html")
fig.write_image("plotly_chart.png")

In [14]:
import requests

token = 'G68BgzaWHmFGUJUm4VAnyHS1w3oRFu2n1fgBiCqkTHE'

# 要發送的訊息
message = '這是用 Python 發送的訊息與圖片'

# HTTP 標頭參數與資料
headers = { "Authorization": "Bearer " + token }
data = { 'message': message }

# 要傳送的圖片檔案
image = open('plotly_chart.png', 'rb')
files = { 'imageFile': image }

# 以 requests 發送 POST 請求
requests.post("https://notify-api.line.me/api/notify",
    headers = headers, data = data, files = files)

<Response [200]>